In [ ]:
import random
from collections import deque
import gym
import gymnasium
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create environment
env = gym.make("MountainCar-v0")
state_size = env.observation_space.shape[0]  # 4
action_size = env.action_space.n
state_size, action_size

In [ ]:
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super().__init__()
        self.fc1 = nn.Linear(state_size, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, action_size)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = F.relu(self.fc3(x))
        return self.fc4(x)

In [ ]:
class ReplayBuffer:
    def __init__(self, buffer_size, batch_size):
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size

    def add(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def sample(self):
        experience = random.sample(self.memory, self.batch_size)
        states = torch.from_numpy(np.vstack([e[0] for e in experience])).float().to(device)
        actions = torch.from_numpy(np.vstack([e[1] for e in experience])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e[2] for e in experience])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in experience])).float().to(device)
        dones = torch.from_numpy(np.vstack([e[4] for e in experience]).astype(np.uint8)).float().to(device)
        return (states, actions, rewards, next_states,dones)


    def __len__(self):
        return len(self.memory)

In [ ]:
class Agent:
    def __init__(self, state_size, action_size, buffer_size=100000, batch_size=64, gamma=0.995, lr=1e-3, tou=1e-2):
        self.state_size = state_size
        self.action_size = action_size
        self.batch_size = batch_size
        self.gamma = gamma
        self.lr = lr
        self.tou = tou

        self.qnetwork_local = QNetwork(state_size, action_size).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.lr)
        self.memory = ReplayBuffer(buffer_size, batch_size)
        self.t_steps = 0
        self.update_every = 4

    def act(self, state, epsilon=0.0):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)

        self.qnetwork_local.eval()
        with torch.no_grad():
            q_values = self.qnetwork_local(state)
        self.qnetwork_local.train()
        if random.random() > epsilon:
            return torch.argmax(q_values).item()
        else:
            return np.random.choice(np.arange(self.action_size))

    def step(self, state, action, reward, next_state, done):
        self.memory.add(state, action, reward, next_state, done)
        self.t_steps = (self.t_steps + 1) % self.batch_size
        if self.t_steps % self.update_every == 0 and len(self.memory) > self.batch_size:
            experiences = self.memory.sample()
            self.learn(experiences)

    def learn(self, experiences):
        states, actions, rewards, next_states, dones = experiences
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        Q_targets = rewards + (self.gamma * Q_targets_next * (1 - dones))

        Q_expected = self.qnetwork_local(states).gather(1, actions)
        loss = F.mse_loss(Q_expected, Q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.soft_update(self.qnetwork_local, self.qnetwork_target)

    def soft_update(self, local_model, target_model):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(target_param.data * (1.0 - self.tou) + local_param.data * self.tou)

In [ ]:
n_episodes = 1000
max_t = 1000
epsilon_start = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
scores = []
scores_window = deque(maxlen=100)
agent = Agent(state_size, action_size)
eps=epsilon_start

## 1. Inputs
- **state**: Current observation [position, velocity] before taking the action.

- **next_state**: Next observation [position, velocity] after taking the action.

- **reward_env**: Default reward from the environment (usually -1 per step in MountainCar).

- **done**: Whether the episode is finished (reached goal or timeout).

## 2. Components of the Shaped Reward

#### (A) velocity_reward = 5 * abs(velocity)
- Purpose: Encourages the car to move fast, regardless of direction.

- MountainCar requires building momentum by swinging back and forth.

- Without this, the car might just stay still (since the default reward is -1 per step).

- abs(velocity): Rewards speed in both left (-) and right (+) directions.

- 5: A scaling factor (hyperparameter). Higher = more focus on speed.

#### (B) progress_reward = 10 * (next_state[0] - state[0]) if next_state[0] > state[0] else 0
- Purpose: Rewards the car for moving right (toward the goal).

- The goal is at position = 0.5 (right side).

- This incentivizes progress, not just random swinging.

- next_state[0] - state[0]: Measures how much the car moved right.

- 10: Scaling factor. Higher = more focus on rightward movement.

#### (C) goal_bonus = 100 if position >= 0.5 else 0
- Purpose: Huge bonus for reaching the goal.

- The default reward (-1 per step) is too sparse.

- Without this, the agent might never discover the goal.

- 100: Arbitrary large value to make the goal extremely attractive.

#### (D) shaped_reward = reward_env + velocity_reward + progress_reward + goal_bonus
- Combines all rewards into a single scalar value.

- reward_env: Usually -1 (punishment for taking too long).

#### 3. How This Helps the Agent Learn
- First, it learns to swing back and forth (due to velocity_reward).

- Then, it learns to swing harder to the right (due to progress_reward).

- Finally, it discovers the goal (due to goal_bonus).



In [ ]:
def get_shaped_reward(state, next_state, reward_env, done):
    position = next_state[0]
    velocity = next_state[1]

    velocity_reward = 5 * abs(velocity)


    progress_reward = 10 * (next_state[0] - state[0]) if next_state[0] > state[0] else 0

    goal_bonus = 100 if position >= 0.5 else 0

    shaped_reward = reward_env + velocity_reward + progress_reward + goal_bonus
    return shaped_reward

In [ ]:
for episode in range(1, n_episodes + 1):
    state, _ = env.reset()
    total_reward = 0
    max_position = -1.2

    for t in range(max_t):
        action = agent.act(state, eps)
        next_state, reward_env, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        reward = get_shaped_reward(state, next_state, reward_env, done)
        agent.step(state, action, reward, next_state, done)

        state = next_state
        total_reward += reward
        max_position = max(max_position, next_state[0])  # Track position

        if done:
            break

    scores.append(total_reward)
    scores_window.append(total_reward)
    eps = max(epsilon_min, epsilon_decay * eps)

    print(
        f"Episode {episode}, "
        f"Score: {total_reward:.2f}, "
        f"Avg100: {np.mean(scores_window):.2f}, "
        f"Epsilon: {eps:.2f}, "
        f"Max Position: {max_position:.4f}"
    )

    if np.mean(scores_window) >= -110:  # If the agent is close to solving it
        print(f"✅ Solved in {episode} episodes! Avg score: {np.mean(scores_window):.2f}")
        torch.save(agent.qnetwork_local.state_dict(), 'mountaincar.pth')
        break
env.close()


In [ ]:
trained_agent = Agent(state_size, action_size)
trained_agent.qnetwork_local.load_state_dict(torch.load("mountaincar.pth"))
trained_agent.qnetwork_local.to(device)

import gymnasium as gym
import base64
import imageio
import os
from IPython.display import HTML, display


def show_video_of_model(agent, env_name="MountainCar-v0", output_path="video.mp4"):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []

    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

    env.close()

    # Save video
    imageio.mimsave(output_path, frames, fps=30)
    print(f"✅ Video saved to: {os.path.abspath(output_path)}")


def show_video(video_path="video.mp4"):
    if os.path.exists(video_path):
        with open(video_path, 'rb') as f:
            video = f.read()
        encoded = base64.b64encode(video).decode('ascii')
        display(HTML(f'''
            <video alt="test" autoplay loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{encoded}" type="video/mp4" />
            </video>'''))
    else:
        print(f"❌ Could not find video at {video_path}")


# 🔽 Example usage:
output_file = "/Artificial/AI/cartpole_trained.mp4"
show_video_of_model(agent, env_name="MountainCar-v0", output_path=output_file)
show_video(output_file)
